## Build Functions for ELT

In [1]:
# Install Requirments (Updated on 10/2/2024, streamlit 1.39.0)
# !pip3 install -r requirements.txt

In [2]:
import pandas as pd
import numpy as np
from IPython.display import display
from yfinance import Ticker
from pykalman import KalmanFilter

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

from sklearn.pipeline import Pipeline

from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

from sklearn.model_selection import cross_val_predict

from datetime import datetime
import pytz

import pprint

from src import functions as f

%load_ext autoreload
%autoreload 2

### Download, Transform, and Modeling All in One

In [3]:
# manufactoring suppliers 
# 'lrcs', 'klac', ''

In [6]:
f.predictions('cat')

CAT 5m Interval Timestamp: 2024-10-30 17:53:13 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,up,up,up
kelly_1:2.5,0.125,0.621112,0.620741,0.29226
prob_up,0.375,0.729366,0.7291,0.494471
prob_static,0.25,0.216725,0.19619,0.273757
prob_down,0.375,0.053909,0.074709,0.231772
precision,0.766744,0.779703,0.756501,0.662234
recall,0.815725,0.773956,0.786241,0.611794
f1,0.790476,0.776819,0.771084,0.636015
support,"[407.0, 407.0, 406.0]","[407.0, 407.0, 406.0]","[407.0, 407.0, 406.0]","[407.0, 407.0, 406.0]"


CAT 15m Interval Timestamp: 2024-10-30 17:53:21 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,static,static,static
kelly_1:2.5,0.3,0.459472,0.61764,0.432007
prob_up,0.5,0.059047,0.093027,0.170602
prob_static,0.25,0.613908,0.726886,0.594291
prob_down,0.25,0.327045,0.180088,0.235107
precision,0.765823,0.716981,0.721854,0.541667
recall,0.75625,0.721519,0.689873,0.658228
f1,0.761006,0.719243,0.705502,0.594286
support,"[160.0, 158.0, 157.0]","[160.0, 158.0, 157.0]","[160.0, 158.0, 157.0]","[160.0, 158.0, 157.0]"


CAT 30m Interval Timestamp: 2024-10-30 17:53:25 


...

,XGBoost,GradientBoosting,RandomForest,KNN
prediction,down,down,down,down
kelly_1:2.5,0.475,0.995892,0.991652,0.912874
prob_up,0.0,0.000256,0.001388,0.019632
prob_static,0.375,0.002678,0.004575,0.042602
prob_down,0.625,0.997066,0.994037,0.937767
precision,0.780645,0.760656,0.729904,0.735426
recall,0.828767,0.794521,0.777397,0.561644
f1,0.803987,0.777219,0.752902,0.636893
support,"[293.0, 292.0, 292.0]","[293.0, 292.0, 292.0]","[293.0, 292.0, 292.0]","[293.0, 292.0, 292.0]"


CAT 1wk Interval Timestamp: 2024-10-30 17:53:40 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,up,down,down,down
kelly_1:2.5,0.3,0.706876,0.64225,0.435821
prob_up,0.5,0.006404,0.127633,0.097594
prob_static,0.25,0.20297,0.127902,0.305391
prob_down,0.25,0.790626,0.744465,0.597015
precision,0.864407,0.746032,0.774194,0.714286
recall,0.87931,0.824561,0.842105,0.614035
f1,0.871795,0.783333,0.806723,0.660377
support,"[58.0, 58.0, 57.0]","[58.0, 58.0, 57.0]","[58.0, 58.0, 57.0]","[58.0, 58.0, 57.0]"


CAT 1mo Interval Timestamp: 2024-10-30 17:53:42 


,XGBoost,GradientBoosting,RandomForest,KNN
prediction,static,static,static,static
kelly_1:2.5,0.3,0.944047,0.926453,0.562344
prob_up,0.375,0.030675,0.033457,0.161302
prob_static,0.5,0.960033,0.947467,0.687389
prob_down,0.125,0.009291,0.019077,0.151309
precision,0.807692,0.8,0.782609,0.586207
recall,0.807692,0.769231,0.692308,0.653846
f1,0.807692,0.784314,0.734694,0.618182
support,"[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]","[27.0, 26.0, 27.0]"


### Check Summary Output

In [ ]:
symbol='NVDA'
interval='5m'

# Define Eastern Time Zone
eastern = pytz.timezone('US/Eastern')

# Get current time in Eastern Time Zone
eastern_time = datetime.now(eastern)

# Format the time to include hour, minute, and seconds
time_stamp = eastern_time.strftime('%Y-%m-%d %H:%M:%S')

print(f'DL Time: {time_stamp}')

# f.download(symbol, interval, period)
f.transform(symbol, interval)
curr_prediction, models, feature_names, classification_reports = f.model(symbol, interval)
predictions, prediction_probas = f.make_prediction(models, curr_prediction, feature_names)

f.predictions_summary(predictions, prediction_probas, classification_reports)

In [ ]:
predictions

In [ ]:
classification_reports

### Hyperparameter Searches

In [27]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np

def model(symbol, interval, search_type='none'):
    # Load data
    data = f.load_model_df(symbol, interval)
    data.dropna(inplace=True, axis=0)
    X = data.drop(columns=['direction'], axis=1)
    y = data['direction']
    
    # Print column names to check for issues
    print("Columns in X before preprocessing:")
    print(X.columns)
    
    # Remove duplicate columns
    X = X.loc[:, ~X.columns.duplicated()]
    
    # Check if categorical_features are present in X
    categorical_features = ['day_of_month', 'day_of_week', 'hour_of_day']
    missing_features = [col for col in categorical_features if col not in X.columns]
    if missing_features:
        print(f"Missing categorical features: {missing_features}")
    
    # Store current prediction data (last row)
    curr_prediction = X.iloc[-1].copy()

    # Drop last row from X and y to prevent the model from seeing it
    X = X.iloc[:-1]
    y = y.iloc[:-1]
    
    # Create the categorical transformer
    categorical_transformer = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
    
    # Create the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('cat', categorical_transformer, categorical_features)
        ],
        force_int_remainder_cols=False # This will include all other columns in the transformed output
    )
    
    # Define your models
    models = {
        'XGBoost': XGBClassifier(random_state=42, n_jobs=-1),
        'RandomForest': RandomForestClassifier(random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingClassifier(random_state=42, validation_fraction=0.25, n_iter_no_change=31),
        # 'LightGBM': LGBMClassifier(random_state=42,force_col_wise=True),
        'KNN': KNeighborsClassifier(n_neighbors=7, p=1,weights='distance')
    }
    
    # Hyperparameters to search
    param_grids = {
        'XGBoost': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [3, 5, 7, 9],
            'classifier__learning_rate': [0.01, 0.1, 0.2, 0.3]
        },
        'RandomForest': {
            'classifier__n_estimators': [100, 200, 300],
            'classifier__max_depth': [None, 10, 20, 30],
            'classifier__min_samples_split': [2, 5, 10, 13]
        },
        'GradientBoosting': {
            'classifier__n_estimators': [100, 200, 300, 400],
            'classifier__max_depth': [3, 5, 7, 13],
            'classifier__le||arning_rate': [0.01, 0.1, 0.2, 0.5]
        },
        'KNN': {
            'classifier__n_neighbors': [3, 5, 7, 13],
            'classifier__weights': ['uniform', 'distance'],
            'classifier__p': [1, 2]  # 1: Manhattan, 2: Euclidean
        }
    }
    
    # Create a function to get feature names after transformation
    def get_feature_names_out(column_transformer):
        feature_names = []
        for name, transformer, columns in column_transformer.transformers_:
            if transformer == 'drop' or transformer == 'passthrough':
                if transformer == 'passthrough':
                    feature_names.extend(columns)
                continue
            if hasattr(transformer, 'get_feature_names_out'):
                names = transformer.get_feature_names_out(columns)
                feature_names.extend(names)
            else:
                feature_names.extend(columns)
        return feature_names
    
    # Split data before preprocessing to avoid data leakage
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, stratify=y, test_size=0.2, random_state=42)
    
    # Fit the preprocessor on training data
    preprocessor.fit(X_train)
    
    # Transform training and test data
    X_train_transformed = preprocessor.transform(X_train)
    X_test_transformed = preprocessor.transform(X_test)
    
    # Get feature names after transformation
    feature_names = get_feature_names_out(preprocessor)
    
    # Convert transformed data to DataFrame
    X_train_transformed = pd.DataFrame(X_train_transformed, columns=feature_names)
    X_test_transformed = pd.DataFrame(X_test_transformed, columns=feature_names)
    
    # Transform curr_prediction
    curr_prediction_transformed = preprocessor.transform(
        curr_prediction.to_frame().T)
    curr_prediction_transformed = pd.DataFrame(
        curr_prediction_transformed, columns=feature_names)
    
    for model_name, model in models.items():
        # Create a pipeline with the classifier
        pipeline = Pipeline(steps=[
            ('classifier', model)
        ])
        
        # Get the parameter grid for the current model
        param_grid = param_grids.get(model_name, {})
        
        # Use GridSearchCV or RandomizedSearchCV
        if search_type == 'grid' and param_grid:
            search = GridSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
        elif search_type == 'random' and param_grid:
            search = RandomizedSearchCV(
                pipeline, param_grid, cv=5, scoring='accuracy',
                n_jobs=-1, n_iter=10, random_state=42)
        else:
            search = pipeline
        
        # Fit the model
        search.fit(X_train_transformed, y_train)
        
        # If using search, get the best estimator
        if search_type in ['grid', 'random'] and param_grid:
            best_model = search.best_estimator_
            print(f"Best parameters for {model_name}: {search.best_params_}")
            model = best_model.named_steps['classifier']
        else:
            model = search.named_steps['classifier']
        
        # Store the model
        models[model_name] = model
        
        # Predict on test data
        y_pred = search.predict(X_test_transformed)
        
        # Evaluate the model
        print(f"Model: {model_name}")
        print(classification_report(y_test, y_pred, zero_division=0))
    
    return curr_prediction_transformed, models, feature_names

In [ ]:
curr_prediction, models, feature_names = model('AMD', '5m', 'grid')